In [5]:
import pandas as pd
import numpy as np

from pyecharts import options as opts
from pyecharts.charts import Bar, Timeline,Line, Grid, Tab
from pyecharts.commons.utils import JsCode


# 预处理

In [4]:
video_total_num=pd.read_json('video_total_num.json')

In [6]:
video_total_num

,分区,子分区,时间,视频数
0,动画,MAD·AMV,1月,14235
1,动画,MAD·AMV,2月,29872
2,动画,MAD·AMV,3月,39488
3,动画,MAD·AMV,4月,38694
4,动画,MAD·AMV,5月,32188
...,...,...,...,...
705,资讯,环球,6月,1401
706,资讯,环球,7月,1308
707,资讯,环球,8月,1756
708,资讯,环球,9月,1543


In [7]:
video_total_num.分区.value_counts()

生活    80
音乐    80
游戏    80
舞蹈    60
知识    60
动画    60
鬼畜    50
美食    50
时尚    50
资讯    40
数码    40
影视    40
娱乐    20
Name: 分区, dtype: int64

In [9]:
table=video_total_num.pivot_table('视频数',index='时间',columns='分区',aggfunc='sum')
table=table.reindex([ '1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月','10月'])
table

分区,动画,娱乐,影视,数码,时尚,游戏,生活,知识,美食,舞蹈,资讯,音乐,鬼畜
时间,,,,,,,,,,,,,
1月,97301,195801,178627,27227,44496,970310,825541,112384,69471,31806,3496,245913,5535
2月,162283,283260,309474,42685,62539,1266970,1124565,221752,108727,38670,4527,284623,10527
3月,209795,337176,366417,62587,89932,1500502,1345435,298919,144505,42589,5353,328605,13328
4月,220387,375277,380243,67548,99802,1523390,1271402,332776,148768,47766,5744,361582,13447
5月,193134,342484,366924,63191,94930,1433326,1208360,318671,146241,55321,4071,343718,11733
6月,158796,277190,318817,55746,100211,1365343,1104884,338655,132670,58094,6237,272529,8954
7月,190234,283196,337604,54536,116781,1480767,1128503,400334,143266,60499,7461,304786,10331
8月,217379,307162,350268,56923,140518,1515685,1222269,411064,129057,60173,11870,326974,12986
9月,141196,226557,291392,53020,214591,1273544,1040554,419833,107470,47928,15514,236388,6719


# UP主行为习惯和特征

## 投稿数变化

### 总量变化

In [10]:
table.sum(1)

时间
1月     2807908
2月     3920602
3月     4745143
4月     4848132
5月     4582104
6月     4198126
7月     4518298
8月     4762328
9月     4074706
10月    4454268
dtype: int64

In [40]:
pic1=Line()
pic1.add_xaxis(list(table.sum(1).index))  #一定要是列表
pic1.add_yaxis('',y_axis=[int(value) for value in table.sum(1).values],color=0)    #颜色是RGB十六进制，查“https://www.fontke.com/tool/rgb/”
pic1.set_series_opts(label_opts = opts.LabelOpts(position = 'top'))
pic1.set_global_opts(title_opts=opts.TitleOpts('总量变化',title_textstyle_opts=opts.TextStyleOpts(font_size=25)),
                    tooltip_opts=opts.TooltipOpts(trigger="axis"),  #提示框
                    yaxis_opts=opts.AxisOpts(type_="value",min_=1000000,max_=7000000,axistick_opts=opts.AxisTickOpts(is_show=True),splitline_opts=opts.SplitLineOpts(is_show=True)),
                    xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False),
                    legend_opts=opts.LegendOpts(orient='vertical',pos_right=True)) #boundary_gap是让你的点是对应x轴刻度，而不是在线段中间
    
pic1.render_notebook()

### 增幅对比

In [14]:
relative_diff=round((table.iloc[9,]-table.iloc[0,])/table.iloc[0,],4).sort_values(ascending=False)
relative_diff

分区
时尚    4.0848
资讯    3.9900
知识    2.8015
数码    1.1476
舞蹈    0.9523
美食    0.8522
影视    0.7062
动画    0.6819
游戏    0.4340
生活    0.4021
娱乐    0.2623
鬼畜    0.2182
音乐    0.0690
dtype: float64

In [15]:
absolute_diff=(table.loc['10月']-table.loc['1月']).sort_values(ascending=False)
absolute_diff

分区
游戏    421104
生活    331937
知识    314841
时尚    181759
影视    126155
动画     66349
美食     59203
娱乐     51362
数码     31245
舞蹈     30288
音乐     16960
资讯     13949
鬼畜      1208
dtype: int64

In [16]:
absolute_diff[:3].sum()/absolute_diff.sum()

0.6486321339196773

In [41]:
bar1=Bar()
bar1.add_xaxis(list(relative_diff.index))  #一定要是列表
bar1.add_yaxis("相对增长", [round(value*100,2) for value in relative_diff.values],color='#E79C00')    
bar1.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top',formatter="{c}%"))
bar1.set_global_opts(title_opts=opts.TitleOpts("十个月增长量",
                                             title_textstyle_opts=opts.TextStyleOpts(font_size=25)),
                   yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} %")))

bar2=Bar()
bar2.add_xaxis(list(absolute_diff.index))
bar2.add_yaxis("绝对增长", [int(value)for value in absolute_diff.values],color='#E79C00') 
bar2.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top'))
bar2.set_global_opts(legend_opts=opts.LegendOpts(pos_bottom="45%"))

pic2=Grid()
pic2.add(bar1, grid_opts=opts.GridOpts(pos_bottom="55%"))
pic2.add(bar2, grid_opts=opts.GridOpts(pos_top="55%"))
pic2.render_notebook()

### 各频道时间变化

In [42]:
pic3=Timeline()
for channel in table.columns:
    bar=Bar()
    bar.add_xaxis(list(table.index))  #一定要是列表
    bar.add_yaxis("投稿数", [int(value) for value in table.loc[:,channel].values])    #数字不能是numpy.64这种
    bar.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top'))
    bar.set_global_opts(title_opts=opts.TitleOpts("%s频道2020年1月-10月投稿数"%channel,
                                                 title_textstyle_opts=opts.TextStyleOpts(font_size=25)))
    pic3.add(bar, "%s"%channel)
pic3.render_notebook()

### 各时段频道对比

In [43]:
pic4=Timeline()
for month in list(table.index):
    bar=Bar()
    bar.add_xaxis(list(table.columns))  #一定要是列表
    bar.add_yaxis("投稿数", [int(value) for value in table.loc[month].values],color='#2874B2')    #颜色是RGB十六进制，查“https://www.fontke.com/tool/rgb/”
    bar.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top'))
    bar.set_global_opts(title_opts=opts.TitleOpts("2020年%s各频道投稿数"%month,
                                                 title_textstyle_opts=opts.TextStyleOpts(font_size=25)))
    pic4.add(bar,"%s"%month)
pic4.render_notebook()

### 图表整合

In [23]:
pic1_show=Line({"width": "1800px", "height": "1000px"})
pic1_show.add_xaxis(list(table.sum(1).index))  #一定要是列表
pic1_show.add_yaxis('',y_axis=[int(value) for value in table.sum(1).values],color=0)    #颜色是RGB十六进制，查“https://www.fontke.com/tool/rgb/”
pic1_show.set_series_opts(label_opts = opts.LabelOpts(position = 'top',font_size=25))
pic1_show.set_global_opts(title_opts=opts.TitleOpts('总量变化',title_textstyle_opts=opts.TextStyleOpts(font_size=30)),
                    tooltip_opts=opts.TooltipOpts(trigger="axis"),  #提示框
                    yaxis_opts=opts.AxisOpts(type_="value",min_=1000000,max_=7000000,axistick_opts=opts.AxisTickOpts(is_show=True),splitline_opts=opts.SplitLineOpts(is_show=True),axislabel_opts=opts.TextStyleOpts(font_size = 18)),
                    xaxis_opts=opts.AxisOpts(type_="category", boundary_gap=False,axislabel_opts=opts.TextStyleOpts(font_size = 18)),
                    legend_opts=opts.LegendOpts(orient='vertical',pos_right=True)) #boundary_gap是让你的点是对应x轴刻度，而不是在线段中间

In [25]:
bar1=Bar({"width": "1800px", "height": "1000px"})
bar1.add_xaxis(list(relative_diff.index))  #一定要是列表
bar1.add_yaxis("相对增长", [round(value*100,2) for value in relative_diff.values],color='#E79C00')    
bar1.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top',formatter="{c}%",font_size=25))
bar1.set_global_opts(title_opts=opts.TitleOpts("十个月增长量",
                                             title_textstyle_opts=opts.TextStyleOpts(font_size=30)),
                     xaxis_opts=opts.AxisOpts(axislabel_opts=opts.TextStyleOpts(font_size = 18)),
                     yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} %",font_size = 18)),
                     legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(font_size=25)))

bar2=Bar({"width": "1800px", "height": "1000px"})
bar2.add_xaxis(list(absolute_diff.index))
bar2.add_yaxis("绝对增长", [int(value)for value in absolute_diff.values],color='#E79C00') 
bar2.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top',font_size=25))
bar2.set_global_opts(legend_opts=opts.LegendOpts(pos_bottom="45%",textstyle_opts=opts.TextStyleOpts(font_size=25)),
                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.TextStyleOpts(font_size = 18)),
                    yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 18)))

pic2_show=Grid(init_opts=opts.InitOpts(width="1800px", height="1000px"))
pic2_show.add(bar1, grid_opts=opts.GridOpts(pos_bottom="55%"))
pic2_show.add(bar2, grid_opts=opts.GridOpts(pos_top="55%"))
pic2_show.render_notebook()


In [26]:
pic3_show=Timeline({"width": "1800px", "height": "1000px"})
for channel in table.columns:
    bar=Bar()
    bar.add_xaxis(list(table.index))  #一定要是列表
    bar.add_yaxis("投稿数", [int(value) for value in table.loc[:,channel].values])    #数字不能是numpy.64这种
    bar.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top',font_size=25))  #柱上的那些数字
    bar.set_global_opts(title_opts=opts.TitleOpts("%s频道2020年1月-10月投稿数"%channel,
                                                  title_textstyle_opts=opts.TextStyleOpts(font_size=30)),
                       xaxis_opts=opts.AxisOpts(axislabel_opts=opts.TextStyleOpts(font_size = 18)),
                       yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 18)),
                       legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(font_size=25)))
    pic3_show.add(bar, "%s"%channel)
pic3_show.render_notebook()

In [37]:
pic4_show=Timeline({"width": "1800px", "height": "1000px"})
for month in list(table.index):
    bar=Bar()
    bar.add_xaxis(list(table.columns))  #一定要是列表
    bar.add_yaxis("投稿数", [int(value) for value in table.loc[month].values],color='#2874B2')    #颜色是RGB十六进制，查“https://www.fontke.com/tool/rgb/”
    bar.set_series_opts(label_opts = opts.LabelOpts(is_show = True,position = 'top',font_size=25))
    bar.set_global_opts(title_opts=opts.TitleOpts("2020年%s各频道投稿数"%month,
                                                 title_textstyle_opts=opts.TextStyleOpts(font_size=30)),
                                                 xaxis_opts=opts.AxisOpts(axislabel_opts=opts.TextStyleOpts(font_size = 18)),
                                                 yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size = 18)),
                                                 legend_opts=opts.LegendOpts(textstyle_opts=opts.TextStyleOpts(font_size=25)))
    pic4_show.add(bar,"%s"%month)
pic4_show.render_notebook()

In [29]:
chart1=Tab({"width": "1800px", "height": "1000px"})
chart1.add(pic1_show,'总量变化')
chart1.add(pic2_show,'增幅对比')
chart1.add(pic3_show,'各频道时间变化')
chart1.add(pic4_show,'各时段频道对比')
chart1.render_notebook()

In [30]:
chart1.render('chart1.html')

'E:\\湖大课程\\python数据处理\\大作业\\总\\投稿量分析\\chart1.html'